In [2]:
import pandas as pd
from nltk.tag import StanfordNERTagger
from nltk.tokenize import word_tokenize
from tqdm import tqdm

# 設置 Stanford NER jar 和模型的完整路徑
jar = r"C:\Users\USER\Downloads\NLP-Courses\NLP243\Projects\stanford-ner-2020-11-17\stanford-ner-4.2.0.jar"
model = r"C:\Users\USER\Downloads\NLP-Courses\NLP243\Projects\stanford-ner-2020-11-17\classifiers\english.all.3class.distsim.crf.ser.gz"

# 初始化 Stanford NER Tagger
stanford_ner_tagger = StanfordNERTagger(model, jar, encoding='utf-8')

# 讀取 CSV 文件中的 'translation.en' 列，使用完整路徑
train_df = pd.read_csv(r"C:\Users\USER\Downloads\NLP-Courses\NLP243\Projects\train.csv")
sentences = train_df['translation.en'].dropna().tolist()  # 去除空值並轉換為列表

# 使用 tqdm 查看進度並進行命名實體識別
ner_results = []
for sentence in tqdm(sentences, desc="Processing NER"):
    words = word_tokenize(sentence)
    tagged_words = stanford_ner_tagger.tag(words)
    ner_results.append(tagged_words)

# 將結果轉換為 DataFrame 便於檢視
ner_df = pd.DataFrame({
    "Original Sentence": sentences,
    "NER Tagged": ner_results
})

# 顯示結果
ner_df.head()

# 保存 NER 標記結果到 CSV 文件（可選）
ner_df.to_csv(r"C:\Users\USER\Downloads\NLP-Courses\NLP243\Projects\ner_tagged_train.csv", index=False)


Processing NER:   7%|███▉                                                  | 17077/231266 [5:50:53<73:21:07,  1.23s/it]
Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x000001E1E495F550>>
Traceback (most recent call last):
  File "C:\Users\USER\AppData\Roaming\Python\Python38\site-packages\ipykernel\ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


KeyboardInterrupt: 